In [3]:
def pilot(url):
    
    import re
    import requests
    import pandas as pd 
    import numpy as np 
    import time

    from time import sleep
    from bs4 import BeautifulSoup
    import json
    import random

    import snowflake.connector
    from snowflake.connector.pandas_tools import write_pandas
    from snowflake.connector import connect
    import gspread 
    from oauth2client.service_account import ServiceAccountCredentials
    from df2gspread import df2gspread as d2g

    %load_ext autoreload
    %autoreload 1
    %aimport load_single_df
    
    def search_assets(game_id, platform_id): 
        """
        :param query: product to search
        :type query: callable string

        Fetch data on this asset
        """
        url = f'https://gamevaluenow.com/sold-listings?game_id={game_id}&platform_id={platform_id}&state=cib'

        headers = {
            'accept': 'application/json',
            'accept-encoding': 'utf-8',
            'accept-language': 'en-GB,en;q=0.9',
            'app-platform': 'Iron',
            'referer': 'https://gamevaluenow.com/en-gb',
            'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="102", "Google Chrome";v="102"',
            'sec-ch-ua-mobile': '?0',
            'sec-ch-ua-platform': '"Windows"',
            'sec-fetch-dest': 'empty',
            'sec-fetch-mode': 'cors',
            'sec-fetch-site': 'same-origin',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.62 Safari/537.36',
            'x-requested-with': 'XMLHttpRequest'
        }

        #returns all historical sales data for a given game
        html = requests.get(url=url, headers=headers)
        time.sleep(random.uniform(3,9))
        output = json.loads(html.text)
        return output['listings']
    
    print("search_assets function run complete")
    
    def get_data(url):
        headers = {'accept': '*/*',
            'accept-encoding': 'gzip, deflate, br',
            'accept-language': 'en-US,en;q=0.9',
            #‘app-platform’: ‘Iron’,
            'origin': 'https:/ebay.com',
            'referer': 'https://www.ebay.com/bin/purchaseHistory?item=403405198730',
            'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="102", "Google Chrome";v="102"',
            'sec-ch-ua-mobile': '?0',
            'sec-ch-ua-platform': '"Windows"',
            'sec-fetch-dest': 'empty',
            'sec-fetch-mode': 'cors',
            'sec-fetch-site': 'same-origin',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.62 Safari/537.36'#,
            #‘x-requested-with’: ‘XMLHttpRequest’
              }
        r =  requests.get(url,headers=headers, timeout=10)
        print(r.status_code)
        soup = BeautifulSoup(r.text, 'html.parser')
        return soup
    
    print("get_data function run complete")
    
    def parse_data(soup):
        #finds all game companies
        cats = soup.find_all('div',{'class': 'card collapse-header'})

        #returns titles and links to all platforms
        
        titles = []
        links = []
        for i in range(len(cats)):
            y = cats[i].find_all('a',{'class': ''})
            for system in range(len(y)):
                game = str(y[system])
                title = re.search('>(.+?)<', game)
                title = str(title.group(1))
                titles.append(title)
    
                link = str(y[system])
                link = re.search('/(.+?)"',link)
                link = str(link.group(1))
                links.append(link)
        print('Platforms Found')

        #searches through data of each platform page
        print('Loading Sales Data..........')
        games = []
        for link in range(len(links)):
            try:
                html_doc = requests.get('https://gamevaluenow.com/'+links[link]).text
                data = re.search(r"window\.items = (.*?);", html_doc).group(1)
                data = json.loads(data)
                games.append(data)
            except:
                print(links[link]+' failed')
                pass
        
        #clean data
        games = [j for i in games for j in i]
        games = [{'title':game['title'],'new_price':float(str(game['new_value'].replace(',',""))),'game_id':int(game['game_id']),'platform_id':int(game['platform_id'])} for game in games]
    
        #returns only games where new price >= $1000
        suff_price = []
        for game in games:
            if game['new_price'] >= 1000:
                suff_price.append(game)
        suff_price = [{'game_id':game['game_id'],'platform_id':game['platform_id']} for game in suff_price]
    
        #searches through all sales of each game
        games = [search_assets(game['game_id'],game['platform_id']) for game in suff_price]
        games = [j for i in games for j in i]
    
        #clean data
        for dicts in games:
            dicts['price'] = str(dicts['price'])
            dicts['price'] = dicts['price'].replace(',',"")
            dicts['price'] = float(dicts['price'])
            dicts['vendor'] = dicts['vendor']['name']
    
        #returns data for all sales where price >= $1000
        games = [d for d in games if d['price'] >= 1000]
        pd.set_option('display.max_rows', None)
        games = pd.DataFrame(games)
        games = games[['title','price','vendor','item_number','item_url','end_date']]
    
        return games
    
    print("parse_data function run complete")
    
    all_platforms = get_data(url)
    
    df = parse_data(all_platforms)
    return(df)

In [4]:
url = 'https://gamevaluenow.com/#allPlatforms'
pilot(url)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
search_assets function run complete
get_data function run complete
parse_data function run complete
200
Platforms Found
Loading Sales Data..........
super-nintendo failed
nintendo-switch failed
playstation-3-ps3 failed
playstation-4-ps4 failed
playstation-vita failed


,title,price,vendor,item_number,item_url,end_date
0,Myriad Six Games in One (Nintendo Entertainmen...,2799.99,eBay,282626216303,https://rover.ebay.com/rover/1/711-53200-19255...,2017-09-26
1,Hot Slots for Nintendo NES Complete W/ Case P...,1718.67,eBay,254261503466,https://rover.ebay.com/rover/1/711-53200-19255...,2019-07-04
2,Hot Slots for Nintendo NES Complete W/ Case an...,1429.99,eBay,283360839635,https://rover.ebay.com/rover/1/711-53200-19255...,2019-02-22
3,Hot Slots Unlicensed Nintendo NES Game Super R...,1200.00,eBay,223024667216,https://rover.ebay.com/rover/1/711-53200-19255...,2018-06-28
4,Hot Slots Unlicensed Nintendo NES Game Super R...,1150.00,eBay,253224663426,https://rover.ebay.com/rover/1/711-53200-19255...,2017-11-12
5,Paensian Hot Slots NES Nintendo Cart & Case/Box,1000.00,eBay,142470916599,https://rover.ebay.com/rover/1/711-53200-19255...,2017-08-09
6,Hot Slots for NES Nintendo with original case ...,1099.99,eBay,172673704026,https://rover.ebay.com/rover/1/711-53200-19255...,2017-05-26
7,"Hot Slots Nintendo NES ? Rare ? Original, Nice...",1159.00,eBay,322497116762,https://rover.ebay.com/rover/1/711-53200-19255...,2017-05-07
8,Hot Slots (Nintendo) NES w/ Case Authentic Pan...,1299.99,eBay,281714578607,https://rover.ebay.com/rover/1/711-53200-19255...,2015-06-07
9,NES * HOT SLOTS * Panesian ULTRA RARE!,1100.00,eBay,281618691052,https://rover.ebay.com/rover/1/711-53200-19255...,2015-03-13


In [5]:
with open(r"""/Users/aidanmark/Desktop/Dibbs/Data/private_keys/snf_keys.json""") as f:
    raw_keys = json.load(f)

snf_keys=raw_keys[0]

NameError: name 'json' is not defined

In [ ]:
df.to_csv('gamevaluenow_scrape.csv')

In [ ]:
snf_engine = snowflake.connector.connect(user = str(snf_keys['user']),
                                   password = str(snf_keys['password']),
                                  account = str(snf_keys['account']),
                                    database = str(snf_keys['database']),
                                    warehouse = str(snf_keys['warehouse']),
                                    schema = str(snf_keys['schema']))

In [ ]:
key = "1IMHMhdm6y17tw9rW37icsRfuxaBTlu6cZFGSI6lwkqE"
t_name = "gamevaluenow_scrape"
t_schema = "JPY_CLEAN"
t_db = "PROD_JUPYTER"
engine = snf_engine

In [ ]:
df = load_single_df.fetch_gsheet(key, t_name)
df

In [ ]:
load_single_df.frame_to_snowflake(t_name, df, t_schema, t_db,snf_engine)